# From tabular data to binned data

In [ ]:
import plopp as pp
import scipp as sc

%matplotlib widget

## Overview

Binned data in Scipp is conceptually equivalent to an array of tables.
In other words, it represents an array of records (table rows) sorted into an (often multi-dimensional) array of "bins".
In this tutorial we begin by learning how to setup tabular data appropriate for histogramming and binning with scipp.
The main focus will then be binning the tabular data and basic usage of the resulting binned data.

We will use a file of a simulated neutron-scattering experiment &mdash; at the powder diffractometer [DREAM](https://europeanspallationsource.se/instruments/dream) at the European Spallation Source.
The approach and techniques displayed here are however applicable far more generally and not specific to this scientific area.

## Loading tabular data

We will use a file created by a simulation for a diamond sample using [McStas](http://www.mcstas.org/) and [Geant4](https://geant4.web.cern.ch/).
We can use [scipp.io.load_csv](../../generated/modules/scipp.io.csv.load_csv.rst) to load the file as a dataset.
The column headers of the file encode physical units in the form of `tof [ns]`.
The `'bracket'` header parser understands this format and extracts the units.

In [ ]:
filename = 'https://public.esss.dk/groups/scipp/scipp/1/data_dream_diamond.zip'
ds = sc.io.load_csv(
    filename,
    sep='\t',
    header_parser='bracket',
)
ds

This 1-D dataset represents the tabular data that was read from the file.
In the above table, each row (record) describes an *event*, in this case the detection of a neutron, with its associated metadata such as the detector module or the x, y, and z position.

To histogram or bin data by a column, Scipp must know which columns are metadata and which column holds data values.
The table is actually a table of metadata values for events with an implicit data value of "1 count" each.
To continue we convert this into a data array:

In [ ]:
table = sc.DataArray(
    sc.ones(sizes=ds.sizes, unit='counts'),
    coords={name: column.data for name, column in ds.items()},
)
sc.table(table[:10])

## Histogramming and binning

We are now ready to bin or histogram our data.
Scipp uses the following terminology:

- Binning preserves the original data records as a table associated with each bin.
- Histogramming adds up the values from all contributing records into a single value per bin.

As an initial 1-D example, we will compute a wavelength histogram.
The table can be histogrammed using [sc.hist](https://scipp.github.io/generated/functions/scipp.hist.html) or the equivalent method provided by `DataArray`.
Here we provide a desired bin count (1000).
This will result in equally sized bins covering the full wavelength range:

In [ ]:
histogrammed = table.hist(wavelength=1000)
histogrammed

Instead of specifying bin *count* we may also specify a bin *size*, given by a 0-D variable:

In [ ]:
table.hist(wavelength=sc.scalar(0.02, unit='Angstrom'))

Instead of `hist`, we can use [sc.bin](https://scipp.github.io/generated/functions/scipp.bin.html) (or the equivalent method provided by `DataArray`), which keeps the underlying events and their metadata:

In [ ]:
binned = table.bin(wavelength=1000)
binned

Since we used the same bin count (resulting in the same bin edges) for histogramming and binning, computing the sum of values within each bin (given by `binned.hist()`, a shorthand for `binned.bins.sum()`) yields the same result as histogramming directly.
Therefore only a single line is visible in the following plot:

In [ ]:
bin_sums = binned.hist()  # same as binned.bins.sum()
sc.plot({'histogrammed': histogrammed, 'binned': bin_sums})

While the result of histogramming may appear similar or identical, the internal structure is very different.
The histogrammed data consists of essentially two arrays, one for the values (yellow) and one for the wavelengths (green):

In [ ]:
sc.show(histogrammed)

The top level structure of the binned data is the same, i.e., we have and array of values and an array of wavelengths.
The difference is that each value (bin) stores all contributing table rows:

In [ ]:
sc.show(binned)

### Exercise 1

- Histogram and bin `table` by `z_pos`.
  Plot the results.
- Use `bin` *on the result of the binning from the first bullet* with a *different* value for `z_pos` (for example with more bins), i.e., *not* on the original table `table`.
  Why is this possible?
  
#### Solution

In [ ]:
binned_z = table.bin(z_pos=100)
solution1 = {}
solution1['histogrammed'] = table.hist(z_pos=100)
solution1['binned'] = binned_z.hist()
solution1['binned_high_resolution'] = binned_z.bin(z_pos=300).hist()
sc.plot(solution1)

## Multi-dimensional spatial binning

`bin` can handle multiple dimensions:

In [ ]:
binned_xyz = table.bin(z_pos=31, y_pos=31, x_pos=31)  # 31**3 bins
pp.slicer(
    binned_xyz['z_pos', 20:].hist(), keep=['x_pos', 'y_pos'], norm='log', aspect='equal'
)

Above we can see a cut through the detector assembly, which has the shape of a thick cylinder mantle.

The advantage of binned data over histogrammed data is that metadata for each underlying event is still present.
We can therefore change the binning, or bin in additional dimensions.
For example, we can select the slice containing $z = 0$ and turn it into a higher-resolution cut.

Here we show an example of how, e.g., `scipp.linspace` can be used to create custom bin-edges instead of relying on the edges `bin` determines automatically based on a requested bin-count:

In [ ]:
x_edges = sc.linspace('x_pos', 400, 1500, num=41, unit='mm')
z_slice = binned_xyz['z_pos', sc.scalar(0.0, unit='mm')]
xy_cut = z_slice.bin(y_pos=100, x_pos=x_edges)
xy_cut

In [ ]:
sc.plot(xy_cut.hist().transpose(), aspect='equal')

Using custom bin edges has advantages and disadvantages:

- If a bin-count is provided, `bin` and `histogram` need to compute the minimum and maximum over the coordinate, which adds extra cost to the operation.
- Automatic bin-edges based on a bin-count rarely yield "nice" (rounded) bounds, which may not be desirable for plots.
- Automatic bin-counts avoids two common pitfalls:
  - To obtain, say, 100 bins we must provide 101 edges.
  - Intervals in Scipp are closed on the left but open on the right, i.e., $[...)$.
    If custom bin edges are computed naively from the `max()` of the corresponding coordinate, e.g., using `scipp.linspace`, then the data value at the maximum will *not* be included.

### Exercise 2

- The detector assembly is cylindrical, aligned with the z-axis.
  Compute the distance from the z-axis (from the `x_pos` and `y_pos` coordinates) and store it as a new coordinate in `table`.
  This is the radius in cylinder coordinates.
- Define bin edges for the radius using `scipp.linspace`.
- Bin `table` by `z_pos` and the radius.
- Plot the result.

#### Solution

In [ ]:
radius = sc.sqrt(table.coords['x_pos'] ** 2 + table.coords['y_pos'] ** 2)
table.coords['radius'] = radius
radius_edges = sc.linspace(
    'radius', radius.min(), radius.max() + sc.scalar(1, unit='mm'), num=13
)
binned_zr = table.bin(z_pos=31, radius=radius_edges)
binned_zr.hist().plot()

## Multi-dimensional logical binning

Above we binned according to x, y, and z.
This reflects neither the physics nor the logical structure of the detectors and is generally not very useful.
The original table additionally contains information about the logical structure of the detector array.
In this case it is divided into modules, segments, counters, wires, and strips.
We can use `group` to perform a binning based on discrete values.
The result is 5-D:

In [ ]:
binned_logical = table.group('module', 'segment', 'counter', 'wire', 'strip')
binned_logical

Above we used automatic group setup.
This will create a group for each unique value.
Explicit custom groups (given as a variable) should often be favored in practice, for two reasons:

- If only a group label is provided, `group` needs to determine all unique values in the corresponding coordinate.
  This can be very costly if the input data is large.
- Automatic grouping can be "unstable", in the sense that the actual groups that are created can vary randomly if the input changes.
  This can lead to incompatibilities between grouped data obtained with different inputs.
  If the set of possible coordinate values is known in advance it is therefore beneficial to provide these explicitly.
  
Explicit custom groups can also be used to extract a subset of the data:

In [ ]:
wire1234 = table.group(sc.array(dims=['wire'], values=[1, 2, 3, 4], unit=None))
wire1234.hist().plot()

### Exercise 3

- Group `table` but only by strip and wire.
- Plot the result.

#### Solution

In [ ]:
binned_strip_wire = table.group('strip', 'wire')
print(
    'Neutrons arrive from the "left" in the following figure (low wire index).'
    'They are gradually absorbed so the intensity decreases as we reach deeper '
    'voxel layers:'
)
binned_strip_wire.hist().plot()

## From event-based metadata to bin-based metadata

For each detected neutron our data records the position of the associated voxel.
After the logical grouping above, every bin corresponds to a voxel.

It can be more practical to store the voxel position for every bin (voxel) instead of for every event.
This can be achieved, e.g., by computing the mean for every bin.
Note that in this case all events in a voxel record the same voxel position so this procedure is wasteful &mdash; in practice we may prefer loading the voxel positions directly from a file.

We can also combine the x, y, and z components into a single array of position vectors:

In [ ]:
pos = sc.zeros(sizes=binned_logical.sizes, dtype=sc.DType.vector3, unit='mm')
# We 'pop' the coordinates. If desired they could be kept by using
# __getitem__ instead
pos.fields.x = binned_logical.bins.coords.pop('voxel_x').bins.mean()
pos.fields.y = binned_logical.bins.coords.pop('voxel_y').bins.mean()
pos.fields.z = binned_logical.bins.coords.pop('voxel_z').bins.mean()
binned_logical.coords['position'] = pos
binned_logical

We have removed three event-based coordinates (`voxel_x`, `voxel_y`, and `voxel_z`) and replaced it by a single bin-coordinate, `position`.
Note how the contents of the bins have changed compared to earlier in this tutorial:

In [ ]:
# Extra dimensions sliced out for display purposes, sc.show cannot deal with 5-D data
sc.show(
    binned_logical['strip', 0]['counter', 0]['segment', 0]['module', :5].transpose()
)

Equipped with the position of every voxel, we can compute the number of neutrons counted per voxel and create a 3-D scatter plot.
The "scatter points" correspond to the voxel positions.
In this particular case some voxels had no associated neutrons so the computed position is invalid (`binned.coords['position']` contains `NaN`) and no scatter point is shown:

In [ ]:
counts_per_voxel = binned_logical.hist()
# The following line selects a subset of voxels based on a stride. This is to keep the size
# of the documentation HTML small. You can comment or remove it to plot all voxels.
counts_per_voxel = counts_per_voxel['counter', 0]['strip', ::4]['wire', ::2]
pp.scatter3d(counts_per_voxel, pos='position', pixel_size=20, cbar=True)

We can also inspect an individual component such as a strip:

In [ ]:
strip_counts = binned_logical['strip', 200].hist()
pp.scatter3d(strip_counts, pos='position', pixel_size=10, cbar=True)

### Exercise 4

Above, in [Multi-dimensional logical binning](#Multi-dimensional-logical-binning), we grouped by voxel (based on 5 distinct logical indices) and then computed voxel positions.

- Repeat this without binning by wire, i.e., use only module, segment, counter, and strip.
- Compute the resulting mean positions from event positions analogously to before.
  Alternatively, you can compute these directly from the voxel positions, but note that some voxels have NaN positions, so `nanmean` must be used instead of `mean`.
- Create a scatter plot as before.
  This should yield a rough projection onto a cylinder.
  
#### Solution

In [ ]:
proj = table.group('module', 'segment', 'counter', 'strip')
proj_hist = proj.hist()
proj_hist.coords['proj_pos'] = pos.nanmean('wire')
pp.scatter3d(proj_hist, pos='proj_pos', pixel_size=10, norm='log', cbar=True)

## Binning with edges and groups combined

It is also possible to combine binning and grouping.
Since strips roughly correspond to scattering angle, a plot against wavelength and strip may be useful.

### Exercise 5

- Use group `table` by 'strip' and bin by 'wavelength'.
- Plot the result.

#### Solution

In [ ]:
binned_strip_wavelength = table.group('strip').bin(wavelength=1000)
binned_strip_wavelength.hist().plot(norm='log')